In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bisect
from sklearn.preprocessing import *
import mpl_toolkits.mplot3d 
from sklearn.cluster import KMeans
import ffn as ffn
import empyrical as ep
import ffn as ffn

In [ ]:

ohlcv_data = pd.read_csv("/Users/newuser/Projects/robust_algo_trader/data/gen_oanda_data/GBP_USD_M15_raw_data.csv", parse_dates=['time'])
ohlcv_data = ohlcv_data.set_index('time')
ohlcv_data['year'] = ohlcv_data.index.year
ohlcv_data['month'] = ohlcv_data.index.month
ohlcv_data['day_of_week'] = ohlcv_data.index.dayofweek
ohlcv_data['hour'] = ohlcv_data.index.hour
ohlcv_data['minute'] = ohlcv_data.index.minute
start_date = '2007-01-01'
end_date = '2015-01-01'
ohlcv_data = ohlcv_data[start_date:end_date]
df = ohlcv_data.copy()
df

In [ ]:
df

In [42]:
from sktime.forecasting.model_selection import SlidingWindowSplitter

ONE_DAY = 4*24   
train_size = 80 * ONE_DAY # 80 days 
test_size = 10 * ONE_DAY # 10 days
val_test_horizon = test_size * 2
splitter = SlidingWindowSplitter(window_length=train_size, 
                                 fh=np.arange(1, val_test_horizon + 1),
                                 step_length=test_size)





for i, (train_idx, val_test_idx) in enumerate(splitter.split(df)):
    val_idx = val_test_idx[:test_size]
    test_idx = val_test_idx[test_size:]
    
    df_train = df.iloc[train_idx, :]
    df_val = df.iloc[val_idx, :]
    df_test = df.iloc[test_idx, :]
    
    # get_pips_df(df_train)
    pips_train_df = get_pips_df(df_train)
    time_standard_scaler = StandardScaler().fit(pips_train_df[['day_of_week', 'hour', 'minute']])
    pips_train_df[['day_of_week', 'hour', 'minute']] = time_standard_scaler.transform(pips_train_df[['day_of_week', 'hour', 'minute']])
    filtered_pips_df = cluster_and_filter_pips_df(pips_train_df)
    
    # pips_train_np = pips_train_df.to_numpy()
    
    
    print(f"Window: {i}")
    print(filtered_pips_df)
    if i == 2:
        break
    # print(f"Train: {df_train.shape[0]}")
    # print(f"Val: {df_val.shape[0]}")
    # print(f"Test: {df_test.shape[0]}")
    # print()

Done!
Window: 0
    signal  k_label  calmar_ratio  ulcer_index  annualized_return  \
0        1       85      5.555829     0.008606           0.121860   
1        1       52      3.675285     0.006094           0.073562   
2        1       80      5.468894     0.004298           0.075065   
3        1       90      2.165206     0.013194           0.074816   
4        1       74      3.254112     0.012605           0.075324   
5        1       18      3.605668     0.008857           0.069987   
6        1       49      2.973142     0.008676           0.073314   
7        1       27      4.640753     0.002828           0.058555   
8        1       44      4.351255     0.004414           0.050764   
9        1        6      9.192913     0.001223           0.048531   
10       1       22      7.256462     0.001746           0.043651   
11       1       20      3.493810     0.004143           0.040783   
12       1       57      3.165130     0.004470           0.041347   
13       1       7

In [ ]:
np.random.seed(5)

n_close_points = 24
n_perc_points = 5
dist_measure = 1
future_candles = 10
RETURNS_CONSTANT = 2500
INIT_CAPITAL = 1000
N_CLUSTERS = 100

In [41]:
# Define a function to calculate the ulcer index
def m_ulcer_index(series):
    drawdown = (series - series.cummax()) / series.cummax()
    squared_average = (drawdown**2).mean()
    return squared_average**0.5


# dist_measure
# 1 = Euclidean Distance
# 2 = Perpendicular Distance
# 3 = Vertical Distance
def find_pips(data: np.array, n_pips: int, dist_measure: int):
    pips_x = [0, len(data) - 1]  # Index
    pips_y = [data[0], data[-1]]  # Price
    for curr_point in range(2, n_pips):
        md = 0.0  # Max distance
        md_i = -1  # Max distance index
        insert_index = -1
        # Use a single loop to iterate over all the points
        for i in range(1, len(data) - 1):
            left_adj = bisect.bisect_right(pips_x, i) - 1
            right_adj = left_adj + 1
            # Calculate the distance from the point to the line segment
            d = distance(data, pips_x, pips_y, i, left_adj, right_adj, dist_measure)
            # Update the maximum distance and the insert index
            if d > md:
                md = d
                md_i = i
                insert_index = right_adj
        # Insert the new pip
        pips_x.insert(insert_index, md_i)
        pips_y.insert(insert_index, data[md_i])
    return pips_x, pips_y


# Define a helper function to calculate the distance
def distance(data, pips_x, pips_y, i, left_adj, right_adj, dist_measure):
    time_diff = pips_x[right_adj] - pips_x[left_adj]
    price_diff = pips_y[right_adj] - pips_y[left_adj]
    slope = price_diff / time_diff
    intercept = pips_y[left_adj] - pips_x[left_adj] * slope
    dist_funcs = {
        1: lambda x, y: ((pips_x[left_adj] - x) ** 2 + (pips_y[left_adj] - y) ** 2)
        ** 0.5
        + ((pips_x[right_adj] - x) ** 2 + (pips_y[right_adj] - y) ** 2) ** 0.5,
        2: lambda x, y: abs((slope * x + intercept) - y) / (slope**2 + 1) ** 0.5,
        3: lambda x, y: abs((slope * x + intercept) - y),
    }
    return dist_funcs[dist_measure](i, data[i])


def get_pips_df(sub_df):
    pips_y_list = []
    # loop through the data
    for index in range(n_close_points, len(sub_df)):
        try:
            x_close = sub_df["close"].iloc[index - n_close_points : index].to_numpy()
            pips_x, pips_y = find_pips(x_close, n_perc_points, dist_measure)
            scaled_pips_y = (
                StandardScaler()
                .fit_transform(np.array(pips_y).reshape(-1, 1))
                .reshape(-1)
            )
            pips_y_dict = {f"pip_{i}": scaled_pips_y[i] for i in range(n_perc_points)}
            j = index - 1
            pips_y_dict["year"] = sub_df["year"].iloc[j]
            pips_y_dict["month"] = sub_df["month"].iloc[j]
            pips_y_dict["day_of_week"] = sub_df["day_of_week"].iloc[j]
            pips_y_dict["hour"] = sub_df["hour"].iloc[j]
            pips_y_dict["minute"] = sub_df["minute"].iloc[j]
            pips_y_dict["close"] = sub_df["close"].iloc[j]
            # future features
            pips_y_dict["future_close"] = sub_df["close"].iloc[j + future_candles]
            pips_y_dict["future_return"] = (
                pips_y_dict["future_close"] - pips_y_dict["close"]
            )
            pips_y_dict["future_log_ratio"] = RETURNS_CONSTANT * np.log(
                pips_y_dict["future_close"] / pips_y_dict["close"]
            )
            pips_y_list.append(pips_y_dict)
        except Exception as e:
            print("Done!")
            break
    return pd.DataFrame(pips_y_list)


def cluster_and_filter_pips_df(pips_y_df):
    pips_y_df_np = pips_y_df[
        ["pip_0", "pip_1", "pip_2", "pip_3", "pip_4", "day_of_week", "hour", "minute"]
    ].to_numpy()
    kmeans = KMeans(n_clusters=N_CLUSTERS)
    kmeans.fit(pips_y_df_np)
    pips_y_df["k_label"] = kmeans.labels_
    k_label_returns_df = (
        pips_y_df.groupby("k_label")["future_log_ratio"]
        .sum()
        .sort_values(ascending=False)
    )

    best_k_labels_list = []
    for k_label in k_label_returns_df.index:
        test_pips_y_copy_df = pips_y_df[(pips_y_df["k_label"] == k_label)]
        test_k_label_cumsum = test_pips_y_copy_df["future_log_ratio"].cumsum()
        test_k_label_cumsum = test_k_label_cumsum.reset_index(drop=True)
        if test_k_label_cumsum.iloc[-1] > 0:
            signal = 1
        else:
            signal = 0
            test_k_label_cumsum = -test_k_label_cumsum
        # Add a constant value to the series
        test_k_label_cumsum = test_k_label_cumsum + INIT_CAPITAL
        annualized_return = (
            test_k_label_cumsum.iloc[-1] / test_k_label_cumsum.iloc[0]
        ) - 1
        annualized_std = test_k_label_cumsum.std()
        ulcer_index = m_ulcer_index(test_k_label_cumsum)
        max_drawdown = ffn.calc_max_drawdown(test_k_label_cumsum)
        calmar_ratio = annualized_return / abs(max_drawdown)
        if (
            True
            and calmar_ratio > 2
            # and ulcer_index < 0.15
            # and annualized_return > 0.5
            # and abs(max_drawdown) < 0.3
        ):
            best_k_labels_list.append(
                {
                    "signal": signal,
                    "k_label": k_label,
                    "calmar_ratio": calmar_ratio,
                    "ulcer_index": ulcer_index,
                    "annualized_return": annualized_return,
                    "max_drawdown": max_drawdown,
                }
            )
    return pd.DataFrame(best_k_labels_list)